In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
import csv
import os
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

In [2]:
# set up your dataset folder.
dataset_folder = './data/bank/'

## data preprocessing

In [3]:
if not os.path.exists(dataset_folder):
    print('your folder is not exists')
else:
    with open(dataset_folder + 'bank-full.csv','r',encoding = 'utf-8-sig') as origin_data:
        with open(dataset_folder + 'bank-full-fixed.csv','w',newline='') as output_data:
            spamwriter = csv.writer(output_data)
            spamwriter.writerows([origin_data.readline().rstrip('\n').split(';')])
            for data in origin_data.readlines():
                spamwriter.writerows([data.rstrip('\n').split(';')])

### data encoder

In [4]:
if os.path.exists(dataset_folder+'label_mapping.txt'):
    os.remove(dataset_folder+'label_mapping.txt')
encoder_data = pd.DataFrame()
dataset = pd.read_csv(dataset_folder + 'bank-full-fixed.csv')
for label in dataset.columns:
    if type(dataset[label][1]) != str:
        encoder_data[label] = dataset[label]
    else:
        le = preprocessing.LabelEncoder()
        le.fit(dataset[label])
        encoder_data[label] = le.transform(dataset[label])
        with open(dataset_folder + 'label_mapping.txt','a',encoding='utf-8-sig') as label_mapping:
            label_mapping.write(label+' : '+' , '.join(le.classes_)+'\n')
encoder_data.to_csv(dataset_folder + 'bank-full-fixed-encoder.csv',index=False)

## Decision Trees

In [5]:
dataset = pd.read_csv(dataset_folder + 'bank-full-fixed-encoder.csv')
X = np.array(dataset.iloc[:,:-1])
Y = np.array(dataset['"y"'])

In [6]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [7]:
clf.predict([[58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3]])

array([0], dtype=int64)

In [8]:
clf.predict([[59,0,1,1,0,2343,1,0,2,5,8,1042,1,-1,0,3]])

array([1], dtype=int64)

## ANN

In [9]:
dataset = pd.read_csv(dataset_folder + 'bank-full-fixed-encoder.csv')
X = np.array(dataset.iloc[:,:-1])
Y = np.array(dataset['"y"'])

In [10]:
NN = MLPClassifier()
NN.fit(X,Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [11]:
NN.predict([[59,0,1,1,0,2343,1,0,2,5,8,1042,1,-1,0,3]])

array([0], dtype=int64)

In [12]:
NN.predict([[5,7,1,1,0,2476,1,0,2,5,8,222579,1,-1,0,3]])

array([0], dtype=int64)

In [13]:
NN.predict([[58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3]])

array([0], dtype=int64)